In [1]:
import os
import nltk
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pymorphy2
import seaborn as sns
sns.set_style("whitegrid")
from nltk.tokenize import WhitespaceTokenizer
from nltk.corpus import stopwords
from string import punctuation

# HW1:  Сравнение стилей текстов
### Выполнили:  Булгаков Дмитрий, Тефикова Алие
### Группа ИАД-2

# 1. Загрузка данных

Составьте самостоятельно как минимум две коллекции
текстов разных стилей (например, коллекция текстов в публицистическом
стиле и коллекция текстов в научном стиле). Коллекции текстов
должны быть достаточно большие (порядка 5000 токенов). Посчитайте
количество токенов и типов в каждой коллекции.

### 1.1 Получение  данных из файла

In [2]:
def remove_control_characters(text_string):
    return ''.join(filter(None, text_string.splitlines()))

### 1.1.1 Загрузка художественных текстов (Портрет Дориана Грея,  Оскар Уайльд)

«Портре́т Дориана Гре́я» (англ. The Picture of Dorian Gray) — единственный опубликованный роман Оскара Уайльда. В жанровом отношении представляет смесь романа воспитания с моральной притчей. Существует в двух версиях — в 13 главах (1890 года) и в 20 главах (1891 года). Стал самым успешным произведением Уайльда, более 30 раз экранизировался.

<b>Link:<b> http://lib.ru/WILDE/doriangray.txt

In [3]:
fiction = open('data/dorian_gray.txt', encoding='utf-8').read()
fiction = remove_control_characters(fiction)

### 1.1.2 Загрузка  публицистических текстов (статьи lenta.ru) 

Lenta.ru — одно из ведущих российских новостных интернет-изданий, основанное в 1999 году Антоном Носиком при содействии Фонда эффективной политики. Работает круглосуточно, освещая мировые и внутрироссийские новости.

<b>Link<b>: http://lenta.ru

In [4]:
journalistic = open('data/lentaru.txt', encoding='utf-8').read()
journalistic = remove_control_characters(journalistic)

### 1.1.3 Загрузка  научных текстов (Молодежная Наука 2016)

Материалы Всероссийской научно-практической конференции молодых ученых, аспирантов и студентов, посвященной 150-летию со дня рождения профессора В.Н. Варгина (Пермь, 14-18 марта 2016 года)

<b>Link<b>: http://pgsha.ru/web/science/scientificarticles/

In [5]:
scientific = open('data/perm_conf.txt', encoding='utf-8').read()
scientific = remove_control_characters(scientific)

### 1.1.4 Загрузка  текстов  разговорного стиля (корпус составленный на базе Twitter )

В качестве источника текстов была выбрана платформа микроблогинга Twitter. Современные поисковые системы и имеющиеся в открытом доступе инструменты по сбору текстовых отзывов не позволяют собирать актуальные отзывы и оперативно работать с данными. В связи с этим на основе программного интерефейса API twitter  был разработан программный инструмент для извлечения отзывов об интересующих товарах, услугах,  событиях,  персонах из микроблоггинг-платформы twitter,  который позволяет учитывать время публикации сообщения и авторитетность автора сообщения. Этот инструмент использовался для сбора неразмеченного корпуса. В корпусе содержится более 15 миллионов записей за время с конца ноября 2013 года до конца февраля 2014 года.

<b>Link:<b> http://study.mokoron.com

In [6]:
conversational = open('data/twitter.txt', encoding='utf-8').read()
conversational = remove_control_characters(conversational)

### 1.2 Подсчет токенов и типов

In [7]:
exclude_symbols = set(punctuation + '0123456789'+u'–—'+u'«»'+u'“')

In [8]:
def tokenize(text, exlude_symb):
    text = text.lower()
    text_merged = ''.join(ch for ch in text if ch not in exlude_symb)
    text_tokens = WhitespaceTokenizer().tokenize(text_merged.lower())
    return text_tokens

In [9]:
def print_results(tokens):
    print('N of tokens: ', len(tokens))
    types = nltk.FreqDist(tokens)
    print('N of types:', len(types))

### 1.2.1 Токены и типы для  художественного стиля

In [10]:
fiction_tokens = tokenize(fiction, exclude_symbols)
print_results(fiction_tokens)

N of tokens:  60528
N of types: 18236


### 1.2.2 Токены и типы для публицистического стиля

In [11]:
journalistic_tokens = tokenize(journalistic, exclude_symbols)
print_results(journalistic_tokens)

N of tokens:  13699
N of types: 6015


### 1.2.3 Токены и типы для  научного стиля

In [12]:
scientific_tokens = tokenize(scientific, exclude_symbols)
print_results(scientific_tokens)

N of tokens:  265376
N of types: 57790


### 1.2.4 Токены и типы для  разговорного стиля

In [13]:
conversational_tokens = tokenize(conversational, exclude_symbols)
print_results(conversational_tokens)

N of tokens:  255367
N of types: 56878


### 2. Подсчет частей речи

### 2.1 Подсчет частоты вхождений слов

In [14]:
stop_words = stopwords.words('russian') + [u'это', u'№', u'гсха', u'это', u'оо', u'изз', u'б']
morph_analyzer = pymorphy2.MorphAnalyzer()

In [15]:
def word_freq_count(tokens, morph, stopwords=None):
    lemmata = nltk.FreqDist()
    types = nltk.FreqDist(tokens)
    
    for t in types:
        l = morph.parse(t)[0].normal_form
        if l in lemmata:
            lemmata[l] += types[t]
        else:
            lemmata[l] = types[t]
                
    if (stopwords != None):
        lemmata = lemmata_remove_stopwords(lemmata, stopwords)
    
    return lemmata

In [16]:
def lemmata_remove_stopwords(lem, stopwords):
    lem_no_sw = nltk.FreqDist()
    for l in lem:
        if not l in stopwords:
            lem_no_sw[l] = lem[l]
    return lem_no_sw

In [17]:
def print_lemata_most_common(lem, count=10):
    print('N of lemmata:', len(lem))
    
    for i in lem.most_common(count):
        print(i[0], i[1])

In [18]:
def perform_count_analysis(tokens, morph, stopwords):
    print('Без исключения стоп-слов:\n')
    lem_no_stop = word_freq_count(tokens, morph)
    print_lemata_most_common(lem_no_stop)
    
    print('\n\nИсключая стоп-слова:\n')
    lem_stop = word_freq_count(tokens, morph, stopwords)
    print_lemata_most_common(lem_stop)
    
    

### 2.1.1 Частота вхождений слов для художественного стиля

In [19]:
perform_count_analysis(fiction_tokens, morph_analyzer, stop_words)

Без исключения стоп-слов:

N of lemmata: 11980
и 1914
он 1773
в 1585
я 1477
не 1211
вы 913
что 904
быть 729
на 725
с 689


Исключая стоп-слова:

N of lemmata: 11871
дориан 670
весь 451
человек 296
который 282
свой 277
лорд 253
генри 243
жизнь 226
сказать 225
мочь 196


### 2.1.2 Частота вхождений слов для публицистического стиля

In [20]:
perform_count_analysis(journalistic_tokens, morph_analyzer, stop_words)

Без исключения стоп-слов:

N of lemmata: 4027
в 713
и 299
на 243
что 177
с 174
о 171
год 138
это 135
по 135
не 106


Исключая стоп-слова:

N of lemmata: 3941
год 138
февраль 101
россия 84
который 78
также 51
российский 42
заявить 42
мочь 39
сообщить 38
стать 34


### 2.1.3 Частота вхождений слов для научного стиля

In [21]:
perform_count_analysis(scientific_tokens, morph_analyzer, stop_words)

Без исключения стоп-слов:

N of lemmata: 39007
в 9598
и 9005
на 4623
с 3570
по 2310
год 2245
для 1584
от 1449
что 1329
при 1182


Исключая стоп-слова:

N of lemmata: 38901
год 2245
пермский 1086
который 962
производство 888
являться 812
продукция 804
развитие 727
предприятие 681
хозяйство 674
сельскохозяйственный 651


### 2.1.4 Частота вхождений слов для разговорного стиля

In [22]:
perform_count_analysis(conversational_tokens, morph_analyzer, stop_words)

Без исключения стоп-слов:

N of lemmata: 40435
я 10214
не 7760
в 6501
и 6376
на 4281
что 3477
с 3452
а 3077
весь 3017
ты 2850


Исключая стоп-слова:

N of lemmata: 40319
весь 3017
ещё 1172
хотеть 1125
мочь 916
день 855
сегодня 790
очень 733
просто 669
свой 663
хороший 594


### 2.2 Подсчет количества  слов  с учетом части речи:

Используя любой морфологический процессор, который вам нравится (pymorphy2, mystem), определите к какой части речиотносятся слова из каждой коллекции текстов. При помощи nltk.FreqDist() составьте частотные словари: часть речи – количество слов, к ней относящихся.

In [23]:
parts_of_speech = ['NOUN', 'ADJF', 'ADJS', 'COMP', 'VERB', 
         'INFN', 'PRTF', 'PRTS', 'GRND', 'NUMR', 
         'ADVB', 'NPRO', 'PRED', 'PREP', 'CONJ',
         'PRCL', 'INTJ', 'OTHR']

In [24]:
def count_parts_of_speech(tokens, morph, parts):
    types = nltk.FreqDist(tokens)
    pos_dict = {}
    for t in types:
        word = morph.parse(t)[0].normal_form
        word_pos = morph.parse(word)[0].tag.POS
        
        if (word_pos == None):
            word_pos = 'OTHR'
        
        if (word_pos in pos_dict.keys()):
            pos_dict[word_pos] += types[t]
        else:
            pos_dict[word_pos] = types[t]
        
    return pos_dict

### 2.2.1 Количество слов с учетом части речи для художественного стиля

In [25]:
count_parts_of_speech(fiction_tokens, morph_analyzer, parts_of_speech)

{'ADJF': 7597,
 'ADJS': 61,
 'ADVB': 3885,
 'CONJ': 5897,
 'GRND': 61,
 'INFN': 11352,
 'INTJ': 157,
 'NOUN': 15955,
 'NPRO': 6021,
 'NUMR': 237,
 'OTHR': 160,
 'PRCL': 3369,
 'PRED': 318,
 'PREP': 5314,
 'PRTF': 14,
 'PRTS': 5,
 'VERB': 125}

### 2.2.2. Количество слов с учетом части речи для публицистического стиля

In [26]:
count_parts_of_speech(journalistic_tokens, morph_analyzer, parts_of_speech)

{'ADJF': 1796,
 'ADJS': 2,
 'ADVB': 333,
 'CONJ': 746,
 'GRND': 3,
 'INFN': 1917,
 'INTJ': 11,
 'NOUN': 5985,
 'NPRO': 233,
 'NUMR': 83,
 'OTHR': 302,
 'PRCL': 297,
 'PRED': 23,
 'PREP': 1930,
 'PRTF': 14,
 'PRTS': 1,
 'VERB': 23}

### 2.2.3 Количество слов с учетом части речи для научного стиля

In [27]:
count_parts_of_speech(scientific_tokens, morph_analyzer, parts_of_speech)

{'ADJF': 47272,
 'ADJS': 102,
 'ADVB': 5697,
 'COMP': 2,
 'CONJ': 15460,
 'GRND': 79,
 'INFN': 24950,
 'INTJ': 381,
 'NOUN': 126169,
 'NPRO': 2197,
 'NUMR': 533,
 'OTHR': 8393,
 'PRCL': 2966,
 'PRED': 542,
 'PREP': 30154,
 'PRTF': 156,
 'PRTS': 10,
 'VERB': 313}

### 2.2.4 Количество слов с учетом части речи для разговорного стиля

In [28]:
count_parts_of_speech(conversational_tokens, morph_analyzer, parts_of_speech)

{'ADJF': 27017,
 'ADJS': 345,
 'ADVB': 21903,
 'COMP': 2,
 'CONJ': 24190,
 'GRND': 149,
 'INFN': 43466,
 'INTJ': 2082,
 'NOUN': 65272,
 'NPRO': 20192,
 'NUMR': 696,
 'OTHR': 4153,
 'PRCL': 18255,
 'PRED': 2184,
 'PREP': 24617,
 'PRTF': 40,
 'PRTS': 34,
 'VERB': 770}

Ну что тут сказать?

<img src="picture.jpg">